<a href="https://colab.research.google.com/github/dannnn30/data-mining-_-daniel/blob/main/%5BQuestion%5D_Exercise_Week11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Daniel Oematan - 0706022210060

In [62]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder

!pip install mlxtend==0.23.1

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

you can refer to this article to find out about apriori and association rule:
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here:
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [63]:
# load the data set ans show the first five transaction
df = pd.read_csv('https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv')
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


# Get the set of product that has been purchased


In [64]:
print(set(np.ravel(df.values)))

{'Wine', 'Pencil', 'Bread', 'Diaper', 'Milk', 'Bagel', 'Cheese', 'Meat', 'Eggs', nan}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [65]:
#create an itemset based on the products
itemset = set(np.ravel(df.values))

# encoding the feature
encoding = {product: 1 if product not in ['Milk', 'Bagel', np.nan] else 0 for product in itemset}

print(encoding)

{'Wine': 1, 'Pencil': 1, 'Bread': 1, 'Diaper': 1, 'Milk': 0, 'Bagel': 0, 'Cheese': 1, 'Meat': 1, 'Eggs': 1, nan: 0}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [66]:
# create new dataframe from the encoded features
itemset = pd.unique(df.values.ravel())

encoded_df = pd.DataFrame(0, index=df.index, columns=itemset)

for index, row in df.iterrows():
    for product in row:
        if product in encoded_df.columns:
            encoded_df.at[index, product] = 1

encoded_df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper,Milk,NaN,Bagel
0,1,1,1,1,1,1,1,0,0,0
1,1,1,0,1,1,1,1,1,0,0
2,0,1,1,1,1,0,0,1,1,0
3,0,1,1,1,1,0,0,1,1,0
4,0,1,0,1,0,1,0,0,1,0


In [67]:
# Since, the encoded dataframe consist of the empty column. We will drop the NaN column or u can use the index.
encoded_df = encoded_df.drop(columns=[np.nan])

encoded_df.head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper,Milk,Bagel
0,1,1,1,1,1,1,1,0,0
1,1,1,0,1,1,1,1,1,0
2,0,1,1,1,1,0,0,1,0
3,0,1,1,1,1,0,0,1,0
4,0,1,0,1,0,1,0,0,0


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products.
For this case study, we will min_support=0.2

In [69]:
#Set threshold value untuk digunakan dalam penghitungan support
from mlxtend.frequent_patterns import apriori, association_rules
frequent_itemsets = apriori(encoded_df, min_support=0.2, use_colnames=True)
frequent_itemsets


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.504762,(Bread)
1,0.438095,(Wine)
2,0.438095,(Eggs)
3,0.476190,(Meat)
4,0.501587,(Cheese)
5,0.361905,(Pencil)
6,0.406349,(Diaper)
7,0.501587,(Milk)
8,0.425397,(Bagel)
9,0.244444,"(Wine, Bread)"


The we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [70]:
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Bagel),(Bread),0.425397,0.504762,0.279365,0.656716,1.301042,0.064641,1.442650,0.402687
1,(Wine),(Cheese),0.438095,0.501587,0.269841,0.615942,1.227986,0.050098,1.297754,0.330409
2,(Eggs),(Meat),0.438095,0.476190,0.266667,0.608696,1.278261,0.058050,1.338624,0.387409
3,(Eggs),(Cheese),0.438095,0.501587,0.298413,0.681159,1.358008,0.078670,1.563203,0.469167
4,(Cheese),(Meat),0.501587,0.476190,0.323810,0.645570,1.355696,0.084958,1.477891,0.526414
5,(Meat),(Cheese),0.476190,0.501587,0.323810,0.680000,1.355696,0.084958,1.557540,0.500891
6,(Milk),(Cheese),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148,0.350053
7,(Cheese),(Milk),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148,0.350053
8,"(Eggs, Cheese)",(Meat),0.298413,0.476190,0.215873,0.723404,1.519149,0.073772,1.893773,0.487091
9,"(Eggs, Meat)",(Cheese),0.266667,0.501587,0.215873,0.809524,1.613924,0.082116,2.616667,0.518717


Provide explanation about __antecedent support__, __consequent support__, __support__, __confidence__, __lift__, __leverage__, __conviction__, __conviction__ and the interpretation from the case above (please use text section)

Antecedent support mengukur frekuensi item atau kombinasi item di sisi kiri aturan dalam dataset. Misalnya, "Bagel" muncul dalam 42,5% transaksi. Consequent support, di sisi lain, mengukur frekuensi item di sisi kanan aturan, seperti "Bread" yang tercatat dalam 50,5% transaksi. Support keseluruhan menunjukkan seberapa sering kedua item, seperti "Bagel → Bread," muncul bersama, dengan contoh ini mencapai 27,9% transaksi.

Confidence menggambarkan kemungkinan item di sisi kanan (consequent) muncul jika item di sisi kiri (antecedent) ada. Dalam contoh ini, jika "Bagel" ada, maka "Bread" memiliki peluang 65,7% untuk muncul. Lift membandingkan kemunculan bersama item dengan peluang acak mereka, dan nilai lebih besar dari 1, seperti 1,3, menunjukkan bahwa keduanya lebih sering muncul bersama daripada secara acak. Leverage mengevaluasi seberapa besar peningkatan kemunculan bersama dibandingkan ekspektasi independen, di mana peningkatan sebesar 6,5% tercatat dalam contoh ini.

Conviction menilai kekuatan hubungan antara antecedent dan consequent, dengan nilai lebih besar dari 1 (contohnya 1,44) menunjukkan hubungan yang cukup kuat. Zhang’s metric, metrik tambahan untuk menilai relevansi aturan, menunjukkan nilai 0,40, menandakan aturan ini cukup signifikan meskipun tidak sangat kuat.

Secara keseluruhan, analisis ini mengungkapkan hubungan signifikan antara item tertentu, seperti "Bagel" dan "Bread," yang cenderung muncul bersama lebih sering dibandingkan ekspektasi acak.